In [ ]:
##
# @file This file is part of EDGE.
#
# @author Alexander Breuer (anbreuer AT ucsd.edu)
#
# @section LICENSE
# Copyright (c) 2016-2017, Regents of the University of California
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:
#
# 1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
#
# 2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
#
# 3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
# @section DESCRIPTION
# Derivation of flux solvers.
# Remark: This is for illustrative/debugging purposes. We use the Mathematica script for the implementation.
##

In [ ]:
import sympy
sympy.init_printing(use_unicode=True)

In [ ]:
# create the symbols
laml = sympy.symbols('lambda_l')
mul  = sympy.symbols('mu_l')
rhol = sympy.symbols('rho_l')
lamr = sympy.symbols('lambda_r')
mur  = sympy.symbols('mu_r')
rhor = sympy.symbols('rho_r')

nx   = sympy.symbols('n_x')
ny   = sympy.symbols('n_y')

# define wave speeds
cpl = sympy.sqrt( (laml+2*mul)/rhol )
cpr = sympy.sqrt( (lamr+2*mur)/rhor )
csl = sympy.sqrt( mul / rhol )
csr = sympy.sqrt( mur / rhor )

# create trafos
T = sympy.Matrix([
  [nx*nx,  ny*ny, -2*nx*ny,        0,   0],
  [ny*ny,  nx*nx,  2*nx*ny,        0,   0],
  [nx*ny, -nx*ny,    nx*nx-ny*ny,  0,   0],
  [    0,      0,              0, nx, -ny],
  [    0,      0,              0, ny,  nx] ])

Tm1 = sympy.Matrix([
  [ nx*nx, ny*ny,  2*nx*ny,         0,  0],
  [ ny*ny, nx*nx, -2*nx*ny,         0,  0],
  [-nx*ny, nx*ny,    nx*nx-ny*ny,   0,  0],
  [    0,      0,              0,  nx, ny],
  [    0,      0,              0, -ny, nx] ])

# create Jacobians
Al = sympy.Matrix([
  [0, 0, 0, -(laml+2*mul), 0],
  [0, 0, 0, -laml, 0],
  [0, 0, 0, 0, -mul],
  [-1/rhol, 0, 0, 0, 0],
  [0, 0, -1/rhol, 0, 0]])

Ar = sympy.Matrix([
  [0, 0, 0, -(lamr+2*mur), 0],
  [0, 0, 0, -lamr, 0],
  [0, 0, 0, 0, -mur],
  [-1/rhor, 0, 0, 0, 0],
  [0, 0, -1/rhor, 0, 0]])

# print jacobians and trafo
Al, Ar, T, Tm1

In [ ]:
# define matrix of right and left eigenvectors
R = sympy.Matrix([
  [laml+2*mul, 0, 0, 0, lamr+2*mur],
  [laml, 0, 1, 0, lamr],
  [0, mul, 0, mur, 0],
  [cpl, 0, 0, 0, -cpr],
  [0, csl, 0, -csr, 0]])

R

## Tests for homogeneous equations

In [ ]:
lam = sympy.symbols('lambda')
mu  = sympy.symbols('mu')
rho = sympy.symbols('rho')

cp = sympy.sqrt( (lam+2*mu)/rho )
cp = sympy.sqrt( (lam+2*mu)/rho )
cs = sympy.sqrt( mu / rho )
cs = sympy.sqrt( mu / rho )

A = sympy.Matrix([
  [0, 0, 0, -(lam+2*mu), 0],
  [0, 0, 0, -lam, 0],
  [0, 0, 0, 0, -mu],
  [-1/rho, 0, 0, 0, 0],
  [0, 0, -1/rho, 0, 0]])

B = sympy.Matrix([
  [0, 0, 0, 0, -lam ],
  [0, 0, 0, 0, -(lam+2*mu) ],
  [0, 0, 0, -mu, 0],
  [0, 0, -1 / rho, 0, 0],
  [0, -1 / rho, 0, 0, 0]])

A,B

### First variant: Using normal Jabobian

In [ ]:
An=nx*A+ny*B

Rn = sympy.Matrix([
  [lam+2*mu*nx**2, -2*mu*nx*ny, ny**2, -2*mu*nx*ny, lam+2*mu*nx**2],
  [lam+2*mu*ny**2, 2*mu*nx*ny, nx*nx, 2*mu*nx*ny,  lam+2*mu*ny**2],
  [2*mu*nx*ny, mu*(nx**2-ny**2), -nx*ny, mu*(nx**2-ny**2), 2*mu*nx*ny],
  [nx*cp, -ny*cs, 0, ny*cs, -nx*cp],
  [ny*cp, nx*cs, 0, -nx*cs, -ny*cp]])

evals = sympy.diag( -cp,- cs, 0, cs, cp )

#Pn,Dn = An.diagonalize()
#Pn = sympy.simplify(Pn)

# left flux solver
Fln = 0.5* An + ( Rn*Dn.applyfunc(abs)*Rn**-1 )
# right flux solver
Frn = 0.5* An - ( Rn*Dn.applyfunc(abs)*Ln )

Fln.subs([ (lam, 2), (mu,1), (rho,1), (nx,  1/sympy.sqrt(2)), (ny, 1/sympy.sqrt(2))    ]),\
Frn.subs([ (lam, 2), (mu,1), (rho,1), (nx,  1/sympy.sqrt(2)), (ny, 1/sympy.sqrt(2))    ])
#Rn*Dn*Rn**-1
Rn

In [ ]:
P,D = A.diagonalize()
# left flux solver
Fl = 0.5*T*(A+P*D.applyfunc(abs)*P**-1)*Tm1
# right flux solver
Fr = 0.5*T*(A-P*D.applyfunc(abs)*P**-1)*Tm1

#A,\
#Fl.subs([ (lam, 2), (mu,1), (rho,1), (nx, -0.2),             (ny, sympy.sqrt(1-0.04)) ]),\
#Fr.subs([ (lam, 2), (mu,1), (rho,1), (nx, -0.2),             (ny, sympy.sqrt(1-0.04)) ]),\
#Fl.subs([ (lam, 2), (mu,1), (rho,1), (nx,  0.5),             (ny, -sympy.sqrt(0.75))  ]),\
#Fr.subs([ (lam, 2), (mu,1), (rho,1), (nx,  0.5),             (ny, -sympy.sqrt(0.75))  ]),\
Fl.subs([ (lam, 2), (mu,1), (rho,1), (nx,  1/sympy.sqrt(2)), (ny, 1/sympy.sqrt(2))    ]),\
Fr.subs([ (lam, 2), (mu,1), (rho,1), (nx,  1/sympy.sqrt(2)), (ny, 1/sympy.sqrt(2))    ])

In [ ]:
from sympy.utilities.codegen import codegen

# how to use to matrix outputs in a single code generation? order of out_**** seems random.
# for now we just generate two functions..
codegen( ( "compute2dHomFl", Fln ),
           "C",
           project = "EDGE",
           prefix="compute2dHomFl",
           to_files=True,
           empty=True             )
codegen( ( "compute2dHomFr", Frn ),
           "C",
           prefix="compute2dHomFr",
           to_files=True,
           project = "EDGE",
           empty=True             )

# 3D

In [ ]:
R = sympy.Matrix([
  [laml+2*mul, 0,   0,   0, 0, 0, 0,    0,    lamr+2*mur],
  [laml,       0,   0,   0, 1, 0, 0,    0,    lamr],
  [laml,       0,   0,   0, 0, 1, 0,    0,    lamr],
  [0,          mul, 0,   0, 0, 0, 0,    mur,  0],
  [0,          0,   0,   1, 0, 0, 0,    0,    0],
  [0,          0,   mul, 0, 0, 0, mur,  0,    0],
  [cpl,        0,   0,   0, 0, 0, 0,    0,    -cpr],
  [0,          csl, 0,   0, 0, 0, 0,    -csr, 0],
  [0,          0,   csl, 0, 0, 0, -csr, 0,    0]])

sympy.simplify( R.inv() )